In [40]:
from lxml import etree

In [41]:
_FILE_PATH = 'wmts.xml'
_DEF_NS = {"foo": "http://www.opengis.net/wmts/1.0","ows":"http://www.opengis.net/ows/1.1"}

In [42]:
def get_tile_limits_info(file):
    """Return dict with tile identifier,description, limits for each tile type and zoom option"""
    data = {}
    tree = etree.parse(_FILE_PATH)
    layers = tree.xpath('//foo:Layer', namespaces = _DEF_NS)

    for Layer in layers:
        identifier = Layer.xpath("ows:Identifier" , namespaces = _DEF_NS)[0].text
        description = Layer.xpath("ows:Title" , namespaces = _DEF_NS)[0].text
        #print("Layer id :{0} ==> {1}".format(identifier,description))
        data[identifier]={'description': description}

        TileMatrixLimits_List = Layer.xpath('.//foo:TileMatrixLimits', namespaces = _DEF_NS)
        for TileMatrixLimits in TileMatrixLimits_List:
            TileMatrix = int(TileMatrixLimits.find("foo:TileMatrix", namespaces = _DEF_NS).text)
            MinTileRow = int(TileMatrixLimits.find("foo:MinTileRow", namespaces = _DEF_NS).text)
            MaxTileRow = int(TileMatrixLimits.find("foo:MaxTileRow", namespaces = _DEF_NS).text)
            MinTileCol = int(TileMatrixLimits.find("foo:MinTileCol", namespaces = _DEF_NS).text)
            MaxTileCol = int(TileMatrixLimits.find("foo:MaxTileCol", namespaces = _DEF_NS).text)
            #print("\t{0} {1} {2} {3} {4}".format(TileMatrix, MinTileRow, MaxTileRow,MinTileCol, MaxTileCol))
            data[identifier][TileMatrix] = {'MinTileRow' : MinTileRow,
                                            'MaxTileRow' : MaxTileRow,
                                            'MinTileCol' : MinTileCol,
                                            'MaxTileCol' : MaxTileCol
                                            }
    return(data)

In [43]:
tile_data = get_tile_limits_info(_FILE_PATH)
tile_data['GEOGRAPHICALGRIDSYSTEMS.MAPS.SCAN-EXPRESS.STANDARD'][16]

{'MinTileRow': 21889,
 'MaxTileRow': 35176,
 'MinTileCol': 21264,
 'MaxTileCol': 41027}

In [75]:
def get_scale_denom_info(file):
    """Return dict with tile matrix set identifier, zoom(scale), scale denom(meters/pixel)"""
    data = {}
    tree = etree.parse(_FILE_PATH)
    
    TileMatrixSets = tree.xpath('/foo:Capabilities/foo:Contents/foo:TileMatrixSet', namespaces = _DEF_NS)
    for tilematrixset in TileMatrixSets:
        identifier = tilematrixset.find('ows:Identifier', namespaces = _DEF_NS).text
        supportedcrs = tilematrixset.find('ows:SupportedCRS', namespaces = _DEF_NS).text
        data[identifier] = {}
        data[identifier]['supportedcrs'] = supportedcrs
        TileMatrixes = tilematrixset.findall('foo:TileMatrix', namespaces = _DEF_NS)
        for tilematrix in TileMatrixes:
            scale = tilematrix.find('ows:Identifier', namespaces = _DEF_NS).text
            #Dénominateur d'échelle = résolution / taille pixel ; taille de pixel arbitraire = 0.00028 m
            scale_denom = float(tilematrix.find('foo:ScaleDenominator', namespaces = _DEF_NS).text) * 0.00028 
            data[identifier][scale]=scale_denom
            str_ref = tilematrix.find('foo:TopLeftCorner', namespaces = _DEF_NS).text
            (data[identifier]['X_ref'],data[identifier]['Y_ref']) =  map(float, str_ref.split(" "))
    return data

In [76]:
matrix_set_data = get_scale_denom_info(_FILE_PATH)
matrix_set_data

{'LAMB93': {'supportedcrs': 'EPSG:2154',
  '0': 104579.224549894,
  'X_ref': 0.0,
  'Y_ref': 12000000.0,
  '1': 52277.5323537905,
  '10': 102.0783167832,
  '11': 51.0391448966,
  '12': 25.5195690743,
  '13': 12.7597836936,
  '14': 6.379891636,
  '15': 3.1899457653,
  '16': 1.5949728695,
  '17': 0.7974864315000001,
  '18': 0.3987432149,
  '19': 0.19937160729999998,
  '2': 26135.4870785954,
  '20': 0.09968580370000002,
  '21': 0.04984290179999999,
  '3': 13066.891381800002,
  '4': 6533.2286041135,
  '5': 3266.5595244627,
  '6': 1633.2660045973998,
  '7': 816.6295549859999,
  '8': 408.3139146768,
  '9': 204.1567415109},
 'LAMB93_25cm': {'supportedcrs': 'IGNF:LAMB93',
  '0': 131072.0,
  'X_ref': 0.0,
  'Y_ref': 12000000.0,
  '1': 65536.0,
  '10': 128.0,
  '11': 64.0,
  '12': 32.0,
  '13': 16.0,
  '14': 8.0,
  '15': 4.0,
  '16': 2.0,
  '17': 1.0,
  '18': 0.5,
  '19': 0.25,
  '2': 32768.0,
  '3': 16384.0,
  '4': 8192.0,
  '5': 4096.0,
  '6': 2048.0,
  '7': 1024.0,
  '8': 512.0,
  '9': 256.0}